In [7]:
!pip install scikit-learn xgboost pandas matplotlib

In [3]:
!gcloud services enable ml.googleapis.com
!gcloud services enable compute.googleapis.com

In [4]:
!mkdir friday_training

In [5]:
!touch ./friday_training/__init__.py

In [8]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import subprocess
from google.cloud import storage
import pandas as pd
import numpy as np

from sklearn import preprocessing
from math import sqrt
#import matplotlib.pyplot as plt
#import seaborn as sns
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve

# Fill in your Cloud Storage bucket name
BUCKET_ID = 'mlspec_demo2'



ImportError: No module named 'xgboost'

In [44]:
#train = pd.read_csv('train.csv')
train = pd.read_csv('gs://mlspec_demo2/Data/train.csv')
test = pd.read_csv('gs://mlspec_demo2/Data/test.csv')

In [45]:
train.columns

Index([u'User_ID', u'Product_ID', u'Gender', u'Age', u'Occupation',
       u'City_Category', u'Stay_In_Current_City_Years', u'Marital_Status',
       u'Product_Category_1', u'Product_Category_2', u'Product_Category_3',
       u'Purchase'],
      dtype='object')

In [46]:
numeric_features = train.select_dtypes(include=[np.number])
numeric_features.dtypes

User_ID                 int64
Occupation              int64
Marital_Status          int64
Product_Category_1      int64
Product_Category_2    float64
Product_Category_3    float64
Purchase                int64
dtype: object

In [47]:
# categorical columns that i chose
categorical_columns = ["Product_ID", "Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
                       "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]

In [48]:
# Join Train and Test Dataset so it can be cleaned all at once
train['source']='train'
test['source']='test'

data = pd.concat([train,test], ignore_index = True, sort = False)

In [49]:
#imput the value -2 for NaN since NaN most likely means that the person did not buy products from these categories
data["Product_Category_2"]= \
data["Product_Category_2"].fillna(-2.0).astype("float")

In [50]:
data["Product_Category_3"]= \
data["Product_Category_3"].fillna(-2.0).astype("float")

In [51]:
#Get index of all columns with product_category_1 equal 19 or 20 from train and remove since not populated
condition = data.index[(data.Product_Category_1.isin([19,20])) & (data.source == "train")]
data = data.drop(condition)

In [52]:
# convert categorical data to to numerical values.
# convert data in categorical columns to numerical values
encoders = {col:LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    data[col] = encoders[col].fit_transform(data[col])

In [53]:
#create count features of each user
def getCountVar(compute_df, count_df, var_name):
    grouped_df = count_df.groupby(var_name)
    count_dict = {}
    for name, group in grouped_df:
        count_dict[name] = group.shape[0]

    count_list = []
    for index, row in compute_df.iterrows():
        name = row[var_name]
        count_list.append(count_dict.get(name, 0))
    return count_list

In [15]:
data["Age_Count"]  =getCountVar(data, data, "Age")
data["Occupation_Count"]  =getCountVar(data, data, "Occupation")
data["Product_Category_1_Count"]  =getCountVar(data, data,"Product_Category_1")
data["Product_Category_2_Count"]  =getCountVar(data, data, "Product_Category_2")
data["Product_Category_3_Count"]  =getCountVar(data, data,"Product_Category_3")
data["Product_ID_Count"]  =getCountVar(data, data, "Product_ID")

In [16]:
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,Age_Count,Occupation_Count,Product_Category_1_Count,Product_Category_2_Count,Product_Category_3_Count,Product_ID_Count
0,1000001,684,0,0,10,0,2,0,2,0,0,8370.0,train,21185,18171,28791,241829,541656,344
1,1000001,2406,0,0,10,0,2,0,0,5,11,15200.0,train,21185,18171,200699,23575,26283,845
2,1000001,868,0,0,10,0,2,0,11,0,0,1422.0,train,21185,18171,5610,241829,541656,150
3,1000001,844,0,0,10,0,2,0,11,13,0,1057.0,train,21185,18171,5610,78834,541656,496
4,1000002,2769,1,6,16,2,4,0,7,0,0,7969.0,train,30316,35944,162294,241829,541656,283


In [54]:
#Divide into test and train
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

#Export files as mods:
#train.to_csv('gs://mlspec_demo2/Data/train_modified.csv')
#test.to_csv('gs://mlspec_demo2/Data/test_modified.csv')

In [55]:
# remove column we are trying to predict ('Purchase') from features list
train_features = train.drop('Purchase', axis=1)
# create training labels list
train_labels = (train['Purchase'])

In [56]:
# Join Train and Test Dataset
train['source']='train'
test['source']='test'

data = pd.concat([train,test], ignore_index = True, sort = False)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
# load data into DMatrix object
dtrain = xgb.DMatrix(train_features, train_labels)
# train model
bst = xgb.train({}, dtrain, 20)

In [58]:
import datetime
# Export the model to a file
model = 'model.bst'
bst.save_model(model)

# Upload the model to GCS
bucket = storage.Client().bucket(BUCKET_ID)
blob = bucket.blob('{}/{}'.format(
    datetime.datetime.now().strftime('blackfriday_%Y%m%d_%H%M%S'),
    model))
blob.upload_from_filename(model)

In [21]:
train_features.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,672,0,0,10,0,2,0,2,43346,51445
1,1000001,2376,0,0,10,0,2,0,0,4,10
2,1000001,852,0,0,10,0,2,0,11,86772,233149
3,1000001,828,0,0,10,0,2,0,11,12,233148
4,1000002,2734,1,6,16,2,4,0,7,86773,233147


In [42]:
%%writefile ./blackfriday_training/__init__.py
# Note that __init__.py can be an empty file.

Writing ./census_training/__init__.py


In [1]:
%env PROJECT_ID 'ml-sandbox-1-191918'
%env BUCKET_ID 'mlspec_demo2'
%env MODEL_NAME 'model.bst'
%env VERSION_NAME <YOUR_VERSION>
%env REGION <ML_ENGINE_REGION>
%env INPUT_DATA_FILE data.json

env: PROJECT_ID='ml-sandbox-1-191918'
env: BUCKET_ID='mlspec_demo2'
env: MODEL_NAME='model.bst'
env: VERSION_NAME=<YOUR_VERSION>
env: REGION=<ML_ENGINE_REGION>
env: INPUT_DATA_FILE=data.json


In [2]:
! gcloud config set project 'ml-sandbox-1-191918'

Updated property [core/project].


In [59]:
! gsutil cp ./model.bst gs://mlspec_demo2/model.bst

Copying file://./model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 91.5 KiB/ 91.5 KiB]                                                
Operation completed over 1 objects/91.5 KiB.                                     


In [45]:
! gcloud ml-engine models create 'BlackFriday' --regions 'us-east4'

Created ml engine model [projects/ml-sandbox-1-191918/models/BlackFriday].


In [41]:
! curl -X POST -H "Content-Type: application/json" \
   -d '{"name": "'v1'", "deploymentUri": "gs://mlspec_demo2/", "runtimeVersion": "1.4", "framework": "XGBOOST"}' \
   -H "Authorization: Bearer `gcloud auth print-access-token`" \
    https://ml.googleapis.com/v1/projects/ml-sandbox-1-191918/models/BlackFriday/versions

{
  "error": {
    "code": 409,
    "message": "Field: version.name Error: A version with the same name already exists.",
    "status": "ALREADY_EXISTS",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.BadRequest",
        "fieldViolations": [
          {
            "field": "version.name",
            "description": "A version with the same name already exists."
          }
        ]
      }
    ]
  }
}


In [ ]:
!gcloud ai-platform local pr